### <font color=red>ALOHA Network Simulation </font>

The following code implements the simulation of a 2-node ALOHA network. It is based on a problem posed in Dr. Norman Matloff's open textbook *From Algorithms to Z-Scores: Probabilistic and Statistical Modeling in Computer Science*, which can be downloaded from [here](http://heather.cs.ucdavis.edu/probstatbook). 

The salient features of the network are follows:

* The nodes share a common channel for sending messages
* Only one message can be sent through various time segments known as epochs, denoted by $k=0, 1, 2, 3 \ldots n$
* Only one node can send a message through the channel at the end of an epoch. If both nodes try to send a message at the same time, collision occurs and no message gets sent
* A node is said to be active if it has a message to send
* At epoch $k = 0$, both nodes have messages to send. Hence, both nodes start out active
* X is a random variable that holds the total number of active nodes at the end of epoch $k$. Hence, $X_k = 2$ for $k=0$
* Each node sends a message with probability $p=0.4$ if there is a message to send
* If a node does not have a message to send, it generates a new message in the middle of an epoch with probability $q=0.8 

The following code simulates the ALOHA network for two epochs, i.e. $k_1 = 1, 2 $, and calculates the following probabilities:
\begin{align}
1. & P(X_1=2)\\
2. & P(X_2=2)\\
3. & P(X_1=2 \cap X_2=2)\\
4. & P(X_2=2 \mid X_1=2)
\end{align}




In [19]:
# import the requird libraries
import numpy as np

# declare some "constants"
n_simulations = 1000000
n_nodes = 2
n_epochs = 3
p = 0.4
q = 0.8

# seed the random number generator
np.random.seed(5)

# create and popuplate an ndarray with probabilities for a new node to generaten a message
prob_gen_msg = np.random.random((n_simulations, n_epochs, n_nodes))

# generate (or not) the message based on prob_gen_msg
gen_msg = (prob_gen_msg <= q).astype(int)

# create and popuplate an ndarray with probabilities for a new node to send a message
prob_send_msg = np.random.random((n_simulations, n_epochs, n_nodes)) 

# send (or not) the message based on prob_send_msg
send_msg = (prob_send_msg <= p).astype(int)

# create and popuplate an ndarray with 0s to hold node states at the end of each epoch
node_states = np.zeros((n_simulations, n_epochs, n_nodes)) 

# since both nodes start out active, set both nodes to 1 at epoch k = 0
node_states[:, 0, :] = 1

During each epoch, implement the followings:

* For all nodes that were inactive at the end of the last epoch, try to generate new messages with probability q. Update the state of the nodes to active if messages were generated for them
* For all nodes that are active, try to send messages with probability p. If there is only one node that tries to send a message, update the state of that node to inactive.

In [5]:
# start simulations
for i in range(n_simulations):
    for j in range(1, n_epochs):
        for k in range(n_nodes):        
            if node_states[i, j - 1, k] == 0:
                node_states[i, j, k] = gen_msg [i, j, k]
            else:
                node_states[i, j, k] = 1 # node was active in previous epoch. Just copy it        
        
        if np.sum(send_msg [i, j, :]) == 1:
            for k in range(n_nodes): 
                if send_msg [i, j, k] == 1:
                    node_states[i, j, k] = 0
                    break # there should be only one node that sends messages                   
           

Now that the simulations have been completed, capture $X_k$ where $k=0 \ldots n$.

In [6]:
# create and populate an ndarray to hold events X_1=2 and X_2=2, and initialise them with 0's
X_k = np.zeros((n_simulations, n_epochs)) 

# populate X_1 and X_2, summing active nodes during each episode
X_k = np.sum(node_states[:, 1:, :], axis=2) # ignore epoch 0

In [7]:
# Calculate P(X_1 == 2)
np.mean(X_k[:, 0]==2)

0.52040399999999998

In [8]:
# Calculate P(X_2 == 2)
np.mean(X_k[:, 1] == 2)

0.47081299999999998

In [20]:
# Calculate P(X_1 = 2) and P(X_2 = 2) -- I am sure there is a better way of doing it
# np.sum(X_k[:, 0] == 2 and X_k[:, 1] == 2)
K_1, K_2 = (X_k[:, 0]  == 2), (X_k[:, 1] == 2) 
print(sum(1 for i, j in zip(K_1, K_2) if i==True and j==True)/len(K_1))

0.270993


In [21]:
# Calculate P(X2=2 | X1=2)
print(sum(1 for i, j in zip(K_1, K_2) if i==True and j==True)/sum(K_1==True))  

0.520735812945


In the open textbook mentioned above, these probabilities are worked out analytically.